In [1]:
import pandas as pd
from pymongo import MongoClient
import os

In [2]:
path = "/Users/khaxis/workspace/image_tags/wordnet"
path_is_a = os.path.join(path, 'wordnet.is_a.txt')
path_words = os.path.join(path, 'words.txt')
path_image_urls = "/Users/khaxis/workspace/image_tags/fall11_urls.txt"
path_image_urls = "/Users/khaxis/workspace/image_tags/t"

In [3]:
wordnet_is_a_pd = pd.read_csv(path_is_a, sep=' ', names=['NId', 'subNId'])
wordnet_words_pd = pd.read_csv(path_words, sep='\t', names=['NId', 'Description'])
image_urls = pd.read_csv(path_image_urls, sep='\t', names=['IId', 'Url'])

In [11]:
wordnet_is_a_pd.head(5)

,NId,subNId
0,n02118333,n02119789
1,n02471300,n02478875
2,n02471762,n02473983
3,n02100399,n02100735
4,n02374149,n02390258


In [9]:
wordnet_words_pd.head(5)

,NId,Description
0,n00001740,entity
1,n00001930,physical entity
2,n00002137,"abstraction, abstract entity"
3,n00002452,thing
4,n00002684,"object, physical object"


In [4]:
image_urls['NId'] = image_urls['IId'].apply(lambda l: l.split('_')[0])
image_urls['innerId'] = image_urls['IId'].apply(lambda l: l.split('_')[1])

In [5]:
image_urls.head()

,IId,Url,NId,innerId
0,n00004475_6590,http://farm4.static.flickr.com/3175/2737866473...,n00004475,6590
1,n00004475_15899,http://farm4.static.flickr.com/3276/2875184020...,n00004475,15899
2,n00004475_32312,http://farm3.static.flickr.com/2531/4094333885...,n00004475,32312
3,n00004475_35466,http://farm4.static.flickr.com/3289/2809605169...,n00004475,35466
4,n00004475_39382,http://2.bp.blogspot.com/_SrRTF97Kbfo/SUqT9y-q...,n00004475,39382


Insert data to MongoDB
======================

In [36]:
client = MongoClient()
db = client['image_tags']

In [35]:
res = db.wordnet_is_a.insert_many(wordnet_is_a_pd.to_dict('records'))

In [36]:
res = db.wordnet_words.insert_many(wordnet_words_pd.to_dict('records'))

InvalidStringData: strings in documents must be valid UTF-8: 'http://www.vastavalo.fi/albums/userpics/12983/thumb_valkoinen_\x96_Kopio.JPG'

In [8]:
res = db.image_urls.insert_many(image_urls.to_dict('records'))

In [37]:
image_urls.shape

(14197122, 4)

In [39]:
image_urls.head().Url.encode('utf8')

Exception KeyboardInterrupt in 'zmq.backend.cython.message.Frame.__dealloc__' ignored


AttributeError: 'Series' object has no attribute 'encode'

Add optimized image_urls
========================

In [37]:
t = image_urls.groupby('NId')['Url'].apply(list).to_frame()#.to_dict('records')
t.reset_index(level=0, inplace=True)
t.rename(index=str, columns={"Url": "Urls"}, inplace=True)
res = db.image_urls_collection.insert_many(t.to_dict('records'))
